Capítulo 6

6.4 Patrones de unión.

6.4.3 Unión externa 

6.4.3.2 Unión externa por la derecha

Patrón que permite realizar una unión por la izquierda de dos tablas de datos

In [1]:
%%writefile unionDerecha.py
#!/usr/bin/env python
from mrjob.job import MRJob
import re,sys,os

class unionDerecha(MRJob):
    
    def limpiarNombreArchivo(self,archivo):
        encontradaBarra=False
        tamano=len(archivo)
        posicion=tamano-1
        while encontradaBarra==False or posicion==0:
            if archivo[posicion]=="/":
                encontradaBarra=True
                return archivo[posicion+1:tamano]
            else:
                posicion-=1
        if posicion==0:
            return archivo
        
    def mapper_init(self):
        self.namefile=self.limpiarNombreArchivo(os.getenv('map_input_file')) 
        
    def mapper(self,_,line):
        clave=""
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            if self.namefile=="tablaA.csv":
                linea.append(self.namefile)
                clave=linea[0]  
                yield clave,linea
            else:
                linea.append(self.namefile) 
                clave=linea[0] 
                yield clave,linea
                
    def reducer(self,key,values):
        listaA=[]
        listaB=[]
        #Llenamos las dos listas
        for valor in values:
            if valor[len(valor)-1]=="tablaA.csv":
                listaA.append(valor)
            else:
                listaB.append(valor)
        
        ##### Union por la derecha #####
        for valorB in listaB:
            if listaA:
                for valorA in listaA:
                    yield valorA, valorB
            else:
                yield "null",valorB
                
if __name__ == '__main__':
    unionDerecha.run()

Writing unionDerecha.py


In [2]:
!python unionDerecha.py archivos_datos/tablaA.csv archivos_datos/tablaB.csv

["3","3738","New York,NY","tablaA.csv"]	["3","35314","Esto es texto del post","tablaB.csv"]

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\MRSANC~1\AppData\Local\Temp\unionDerecha.mrsanchez.20201203.213017.922934
Running step 1 of 1...
job output is in C:\Users\MRSANC~1\AppData\Local\Temp\unionDerecha.mrsanchez.20201203.213017.922934\output
Streaming final output from C:\Users\MRSANC~1\AppData\Local\Temp\unionDerecha.mrsanchez.20201203.213017.922934\output...
Removing temp directory C:\Users\MRSANC~1\AppData\Local\Temp\unionDerecha.mrsanchez.20201203.213017.922934...



["3","3738","New York,NY","tablaA.csv"]	["3","44002","Esto es mas texto aun","tablaB.csv"]
["5","17556","San Diego, CA","tablaA.csv"]	["5","44921","Aqui digo otra cosa","tablaB.csv"]
["5","17556","San Diego, CA","tablaA.csv"]	["5","44920","Otra cosa que que se dice","tablaB.csv"]
"null"	["8","48675","Decimos muchas cosas, es impresionante","tablaB.csv"]
